In [1]:
%pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 --force-reinstall

Looking in indexes: https://download.pytorch.org/whl/cu124
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB 796.8 kB/s eta 0:52:31
     ---------------------------------------- 0.0/2.5 GB 796.8 kB/s eta 0:52:31
     ---------------------------------------- 0.0/2.5 GB 744.7 kB/s eta 0:56:11
     ---------------------------------------- 0.0/2.5 GB 824.4 kB/s eta 0:50:45
     ---------------------------------------- 0.0/2.5 GB 849.0 kB/s eta 0:49:16
     ---------------------------------------- 0.0/2.5 GB 855.6 kB/s eta 0:48:53
     ---------------------------------------- 0.0/2.5 GB 898.8 kB/s eta 0:46:32
     ---------------------------------------- 0.0/2.5 GB 898.8 kB/s eta 0:46:32
     ---------------------------------------- 0.0/2.5 GB 924.4 kB/s eta 0:45:14
     ---

In [2]:
import torch

def format_pytorch_version(version_str):
    # Example input: "2.0.1+cu118" -> returns "2.0.1"
    return version_str.split('+')[0]

def format_cuda_version(cuda_str):
    # If CUDA is None (CPU-only PyTorch), return "cpu"
    if cuda_str is None:
        return "cpu"
    # Example: "11.8" -> "cu118"
    return "cu" + cuda_str.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

print(f"Installing packages for torch-{TORCH}+{CUDA}...")

%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

%pip install torch_geometric

Installing packages for torch-2.5.1+cu124...
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu124.html
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
     ------------ --------------------------- 0.5/1.7 MB 2.1 MB/s eta 0:00:01
     ------------------ --------------------- 0.8/1.7 MB 1.1 MB/s eta 0:00:01
     ------------------------ --------------- 1.0/1.7 MB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 1.7/1.7 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
     ------ --------------------------------- 0.5/3.5 MB 2.8 MB/s eta 0:00:02
     ------------ --------------------------- 1.0/3.5 MB 2.5 MB/s eta 0:00:01
     ------------------ --------------------- 1.6/3.5 MB 2.5 MB/s eta 0:00:01
     --------------------- ------------------ 1.8/3.5 MB 2.6 MB/s eta 0:00:01
     --------------------------- ------------ 2.4/3.

In [4]:
%pip install scikit-learn imbalanced-learn xgboost

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 2.8 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/11.1 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.1 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.1 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.1 MB 2.7 MB/s eta 0:00:04
   ----- ---------------------------------- 1.6/11.1 MB 1.2 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.1 MB 1.2 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.1 MB 1.2 MB/s eta 0:00:08
   ------- -------------------------------- 2.1/11.1 MB 1.1 MB/s eta 0:00:09
   ------- -------------------------------- 2.1/11.1 MB 1.1 MB/s eta 0:00:09
   -------- ----------------

In [6]:
%pip install optuna

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 1.3 MB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 1.3 MB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 1.6 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Hyperparameter Optimization Citeseer
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 17, 23),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.2, 0.8),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 0.9),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": 1,
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "citeseer_scar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-20 14:06:40,329] A new study created in memory with name: no-name-59b496fa-fd06-49b2-9fcb-6b4dce066342


Running experiment with seed=1:
 - alpha=0.8082072802880657, K=20, layers=1, hidden=64, out=64
 - norm=graphnorm, dropout=0.24060308244857836, margin=0.5952912437772664, lpl_weight=0.6110581554314212
 - ratio=0.34483104877406745, pos_weight=1, aggregation=mean, treatment=removal
{'aggregation': 'mean', 'pos_weight': 1, 'hidden_channels': 64, 'out_channels': 64, 'num_layers': 1, 'alpha': 0.8082072802880657, 'K': 20, 'norm': 'graphnorm', 'dropout': 0.24060308244857836, 'lpl_weight': 0.6110581554314212, 'margin': 0.5952912437772664, 'ratio': 0.34483104877406745, 'treatment': 'removal'}
Epoch 0, Loss: 0.8657, LPL: 1.3863, Contrastive: 0.0479
Epoch 50, Loss: 0.8635, LPL: 1.3863, Contrastive: 0.0422


### Hyperparameter Optimization Citeseer
#### SAR 

In [ ]:
from .train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 15, 25),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean", "max"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "citeseer_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Cora
#### SCAR 

In [ ]:
from .train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean", "max"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "cora_scar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Cora
#### SAR 

In [ ]:
from .train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean", "max"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "cora_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Pubmed
#### SCAR 

In [ ]:
from .train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean", "max"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "pubmed_scar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization Pubmed
#### SAR 

In [ ]:
from .train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean", "max"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "pubmed_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization WikiCS
#### SCAR 

In [ ]:
from .train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean", "max"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "wikics_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


### Hyperparameter Optimization WikiCS
#### SAR 

In [ ]:
from .train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": trial.suggest_float("alpha", 0.1, 1.0),
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": trial.suggest_categorical("norm", [None, "layernorm", "graphnorm"]),
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": trial.suggest_float("margin", 0.1, 1.0),
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.1, 0.5),
        "pos_weight": trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean", "max"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "wikics_sar.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")
